# Dépenses de l'état

On vous donne ici le code. A vous d'en faire l'interprétation.

**NOTE** : c'est le shéma général que vous devez suivre pour toute ACP.

1. calcul de statistiques descriptives et **interprétations**
2. calcul des données de l'ACP
3. étude de l'intertie : répartition de l'inertie pour sur les axes et combien garder d'axes a priori
4. pour les axes d'intérêts :
    1. étude des corrélations des anciens axes avec les nouveaux et **interprétations**
    2. représentations graphiques des projections avec qualités des projections

## Les données

Les données du fichier `depense_etat.csv` décrivent les
 ventilations des dépenses de l'état français de 1872 à 1971. 
Les différents champs sont :
- PVP : pouvoirs publics,
- AGR : agriculture,
- CMI : commerce et industrie,
- TRA : transports,
- LOG : logement et aménagement du territoire,
- EDU : éducation et culture,
- ACO : anciens combattants,
- ACS : actions sociales,
- DEF : défense,
- DET : dette,
- DIV : divers.

In [ ]:
import pandas

depense = pandas.read_csv("./3_ressources/depense_etat.csv", index_col=0)

In [ ]:
depense

In [ ]:
depense.describe()

In [ ]:
# dessins matplotlib + stype seaborn

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

current_palette = sns.color_palette()
sns.palplot(current_palette)

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/visualization.html

fig, ax = plt.subplots(figsize=(7, 7)) 

depense.plot.barh(stacked=True, ax=ax)

# Put the legend out of the figure
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.show() 

## ACP calcul

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [ ]:
X = pandas.DataFrame(scaler.fit_transform(depense), columns=depense.columns, index=depense.index)

In [ ]:
X

In [ ]:
from sklearn.decomposition import PCA 
import numpy

In [ ]:
pca = PCA()
pca.fit(X)

U = numpy.transpose(pca.components_) # vecteurs propres
C = pandas.DataFrame(X.dot(U), index=X.index) # nouvelles coordonnées
I = pandas.DataFrame(numpy.transpose(pca.explained_variance_ratio_))  # information véhiculée

correlations = pandas.DataFrame([[C[facteur].corr(X[column]) for facteur in C] for column in X], index=X.columns)
quality = (C**2).div((X**2).sum(axis=1), axis='index')

## Interprétations

### intertie

nombres d'axes :

In [ ]:
len(X.columns)

Pour des données centrées réduites le % d'inertie par axe est :

In [ ]:
1 / len(X.columns)

In [ ]:
I

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7)) 

I.plot(ax=ax)

plt.title("inertie expliquee")
plt.xlabel("axes principaux")
plt.ylabel("% d'inertie expliquee")

plt.axhline(0.09, color="red")

plt.show() 

On utilise iloc pour sélectionner des ligne et/ou des colonnespar leur position

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html#pandas.DataFrame.iloc

In [ ]:
float(I.iloc[0:3].sum())

A priori on garde 3 axes pour expliquer 75% de l'information.

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7)) 

I.cumsum().plot(ax=ax)

plt.title("inertie expliquée cumulée")
plt.xlabel("axes principaux")
plt.ylabel("% d'inertie expliquee")

plt.axhline(float(I.iloc[0:3].sum()), color="green")

plt.show() 

### Axes 1 et 2

#### Corrélations

In [ ]:
correlations[[0, 1]]

In [ ]:
axe_x = 0
axe_y = 1

fig, ax = plt.subplots(figsize=(7, 7)) 

draw = correlations.plot.scatter(x=axe_x, y=axe_y, ax=ax, color=current_palette[3])


for index, row in correlations.iterrows():
    draw.annotate(str(index), (row[axe_x], row[axe_y]))

draw.add_patch(plt.Circle((0, 0), radius=1, color=(0, 0, 0, .3)))

plt.xlim(-1.2, 1.2)
plt.ylim(-1.2, 1.2)
plt.axvline(0)
plt.axhline(0)

plt.show()

**QUESTION :** Interpétations ?

#### Projections

In [ ]:
axe_x = 0
axe_y = 1

fig, ax = plt.subplots(figsize=(7, 7)) 

draw = C.plot.scatter(axe_x, axe_y, ax=ax, 
                      color=[current_palette[0]]*len(C.index))

for index, row in C.iterrows():
    draw.annotate(str(index), (row[axe_x], row[axe_y]))

plt.axvline(0)
plt.axhline(0)

plt.show() 

**QUESTION :** Interpétations ?

### Qualité de la projection

In [ ]:
# 1er axe
quality[0].sort_values(ascending=False)

In [ ]:
# 2er axe
quality[1].sort_values(ascending=False)

In [ ]:
# 1er et 2ème axes (nommées 0 et 1)
quality[[0, 1]].sum(axis=1).sort_values(ascending=False)

In [ ]:
#angles 
from math import sqrt, acos, pi

quality.iloc[:, [0, 1]].sum(axis=1).apply(lambda x: acos(sqrt(x)) * 180 / pi).sort_values()

interprétations ?

### axe 2 (troisième axe)

In [ ]:
I

In [ ]:
I.loc[2]

* quels sont les corrélations à cet axe ?
* la qualité de projection ?
* interprétations ?
* projections axes 0 et 2 ainsi que 1 et 2

Corrélation axe numéro 2 (3ème axe) :

In [ ]:
correlations[2].sort_values()

In [ ]:
axe_x = 0
axe_y = 2

fig, ax = plt.subplots(figsize=(7, 7)) 

draw = correlations.plot.scatter(x=axe_x, y=axe_y, ax=ax, color=current_palette[3])


for index, row in correlations.iterrows():
    draw.annotate(str(index), (row[axe_x], row[axe_y]))

draw.add_patch(plt.Circle((0, 0), radius=1, color=(0, 0, 0, .3)))

plt.xlim(-1.2, 1.2)
plt.ylim(-1.2, 1.2)
plt.axvline(0)
plt.axhline(0)

plt.show()

Projections axe 0 et 2

In [ ]:
axe_x = 0
axe_y = 2

fig, ax = plt.subplots(figsize=(7, 7)) 

draw = C.plot.scatter(axe_x, axe_y, ax=ax, 
                      color=[current_palette[0]]*len(C.index))

for index, row in C.iterrows():
    draw.annotate(str(index), (row[axe_x], row[axe_y]))

plt.axvline(0)
plt.axhline(0)

plt.show() 

### années spéciales

In [ ]:
quality

Y a-t-il une année qui n'est pas bien représentée par les 3 premiers axes ? Quels axes faudrait-il utiliser pour l'interpréter ?